In [ ]:
import snowflake.snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark import Window
from snowflake.snowpark import functions as F   
from snowflake.snowpark.functions import udf, udtf
from snowflake.snowpark.types import IntegerType, FloatType, StringType, StructField, StructType, DateType
from snowflake.ml.registry import Registry
    
import pandas as pd
import numpy as np

import streamlit as st

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# add version tracking
app_tag = {
    "origin": "sf_sit",
    "name": "hol_sport_predict",
    "version": '{major: 1, minor: 0}'
}

session.query_tag = app_tag
     

In [ ]:
user_name = session.sql('select current_user()').collect()[0][0]


In [ ]:

# let's load our model from the registry into memory

reg = Registry(session=session)

mv = reg.get_model("EURO_24_GAME_PREDICT").default

In [ ]:
# adjustable threashold.
#
# setting at 0.5 introduces too many penalty shootouts

win_threshold = 0.45

# we can also use streamlit controller for this (DON'T DO THIS YET)
#win_threshold = st.slider(label='Threshold', min_value=0.0 ,max_value=1.0 ,value=0.45)

In [ ]:
# lets make a copy of the fixtures, one with original data and then one we'll update as we go along
df_fixture_copy = session.table('fixture')
df_fixture_copy.write.save_as_table(f'fixture_{user_name}',mode='overwrite')

# get list of fixtures for the groups stages
df_round_1 = (
    session.table(f'fixture_{user_name}')
    .select(
        F.col('"MATCH NUMBER"').alias("id"),
        F.col('"ROUND NUMBER"').alias('round'),
        F.to_date(F.col('"DATE"'), "DD/MM/YYYY HH24:MI").alias("date"),
        F.col('"HOME TEAM"').alias('team_1'),
        F.col('"AWAY TEAM"').alias('team_2'),
        F.col('"GROUP"').alias('group')
    )
)

df_round_1

In [ ]:
# call the sproc that will run all our feature engineering code on the games we want to predict
#
# note - the 0 and 36 params refer to fixtures, we are only processing the feature engineering code for fixtures 0 through 36
# but...because we predict for both games twice we end up with 72 rows.

session.call('prep_prediction_data',0,36)

df_pred = session.table('data_for_predictions').order_by('id')

st.dataframe(data=df_pred)

In [ ]:
# run predictions on all the group stage games

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict_proba")
prediction = prediction.with_column('output_game_outcome',F.iff(F.col('predict_proba_1') > win_threshold,1,0))
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.table('predictions').select(
    prediction.col('id'),
    prediction.col('team_1'),
    prediction.col('output_game_outcome')
).order_by('id')

In [ ]:
-- We can also run the prediction in SQL, where we can call our model using:
-- MODEL_NAME!PREDICT_PROBA()

SELECT 
    "ID", 
    "TEAM_1",  
    CAST ("TMP_RESULT"['PREDICT_PROBA_0'] AS DOUBLE) AS "PREDICT_PROBA_0",  
    CAST ("TMP_RESULT"['PREDICT_PROBA_1'] AS DOUBLE) AS "PREDICT_PROBA_1" 
FROM 
(
    WITH SNOWPARK_ML_MODEL_INFERENCE_INPUT AS (
        SELECT  
            *  
        FROM 
            data_for_predictions 
        ORDER BY "ID" ASC
    ),
    MODEL_VERSION_ALIAS AS MODEL EURO2024.PUBLIC.EURO_24_GAME_PREDICT VERSION V_1
                
    SELECT 
        *,
        MODEL_VERSION_ALIAS!PREDICT_PROBA(
            ID, 
            NEUTRAL, 
            TEAM_1_GOAL_DIFF, 
            TEAM_1_TTL_WINS, 
            TEAM_1_TTL_LOSSES, 
            TEAM_2_GOAL_DIFF, 
            TEAM_2_TTL_WINS, 
            TEAM_2_TTL_LOSSES, 
            TEAM_1_VS_TEAM_2_RANK
        ) AS TMP_RESULT
    FROM 
        SNOWPARK_ML_MODEL_INFERENCE_INPUT
)

In [ ]:
# lets take the output of our predictions 
# 
# we restructure this into the groups sorted by points to determine the teams that go through
# 1st & 2nd, along with the 4 teams who ranked the highest in 3rd place (confusing right?) 

session.call('process_group_predictions')

# we should have our final tables for completed group stages

st.dataframe(data=session.table('results_group_stage').filter(F.col('group') == 'Group A').order_by(F.col('group'),F.col('rank').asc()))
st.dataframe(data=session.table('results_group_stage').filter(F.col('group') == 'Group B').order_by(F.col('group'),F.col('rank').asc()))
st.dataframe(data=session.table('results_group_stage').filter(F.col('group') == 'Group C').order_by(F.col('group'),F.col('rank').asc()))
st.dataframe(data=session.table('results_group_stage').filter(F.col('group') == 'Group D').order_by(F.col('group'),F.col('rank').asc()))
st.dataframe(data=session.table('results_group_stage').filter(F.col('group') == 'Group E').order_by(F.col('group'),F.col('rank').asc()))
st.dataframe(data=session.table('results_group_stage').filter(F.col('group') == 'Group F').order_by(F.col('group'),F.col('rank').asc()))
     

In [ ]:
# take the results of the group stage and insert the team names inmto the Round of 16 fixtures
# now we see who will be playing each each in the next phase

session.call('calculate_r16_games')
df_results = session.table(f'fixture_{user_name}').filter(F.col('"ROUND NUMBER"') == 'Round of 16').order_by('"MATCH NUMBER"')

# we can also use Streamlit to show snowpark dataframes (though this is only compatible for Snowflake Notebooks!)
# otherwise use df_results.show(8)

df_results

In [ ]:
# run the feature engineering code
# note the different fixture id parameters

session.call('prep_prediction_data',37,44)
session.table('data_for_predictions').order_by('id')

In [ ]:
# run predictions on all the games

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict_proba")
prediction = prediction.with_column('output_game_outcome',F.iff(F.col('predict_proba_1') > win_threshold,1,0))
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.table('predictions').select(
    prediction.col('id'),
    prediction.col('team_1'),
    prediction.col('output_game_outcome')
).order_by('id')

In [ ]:
# lets process the predictions into a format where we can see the winners
#
# note - since this is a knockout stage, any draws will have to be resolved by a penalty shootout

session.call('process_knockout_predictions')
session.table('results_ko_stage').order_by('match_id')

In [ ]:
# see the quarter final games as a result of the previous round

session.call('calc_knockout_games','Quarter Finals')
session.table(f'fixture_{user_name}').filter(F.col('"ROUND NUMBER"') == 'Quarter Finals').order_by('"MATCH NUMBER"')
     

In [ ]:
# feature engineering code

session.call('prep_prediction_data',45,48)
session.table('data_for_predictions').order_by('id')
     

In [ ]:
# run predictions on all the games

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict_proba")
prediction = prediction.with_column('output_game_outcome',F.iff(F.col('predict_proba_1') > win_threshold,1,0))
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.table('predictions').select(
    prediction.col('id'),
    prediction.col('team_1'),
    prediction.col('output_game_outcome')
).order_by('id')

In [ ]:
# process the predictions to see the winner 

session.call('process_knockout_predictions')
session.table('results_ko_stage').order_by('match_id')

In [ ]:
# calculate the fixtures

session.call('calc_knockout_games','Semi Finals')
session.table(f'fixture_{user_name}').filter(F.col('"ROUND NUMBER"') == 'Semi Finals').order_by('"MATCH NUMBER"')
     

In [ ]:
# feature engineering code

session.call('prep_prediction_data',49,50)
session.table('data_for_predictions').order_by('id')

In [ ]:
# run predictions on all the games

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict_proba")
prediction = prediction.with_column('output_game_outcome',F.iff(F.col('predict_proba_1') > win_threshold,1,0))
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.table('predictions').select(
    prediction.col('id'),
    prediction.col('team_1'),
    prediction.col('output_game_outcome')
).order_by('id')

In [ ]:
# process the predictions to see the winner

session.call('process_knockout_predictions')
session.table('results_ko_stage').order_by('match_id')

In [ ]:
# calculate the final game

session.call('calc_knockout_games','Final')
session.table(f'fixture_{user_name}').filter(F.col('"ROUND NUMBER"') == 'Final').order_by('"MATCH NUMBER"')

In [ ]:
# feature engineering code

session.call('prep_prediction_data',51,51)
result_df = session.table('data_for_predictions').order_by('id')

st.dataframe(data=result_df,use_container_width=True)

In [ ]:
# run predictions on the final 

pred_df = session.table('data_for_predictions').order_by('id')
prediction = mv.run(pred_df, function_name="predict")
prediction.write.save_as_table('predictions',mode='overwrite',table_type='temp')

session.call('process_knockout_predictions')

df = session.table('results_ko_stage').order_by('match_id')
winner = df.collect()[0]['WINNER']

st.dataframe(data=session.table('results_ko_stage').order_by('match_id'))
st.header(f"{winner} wins the Euro 2024 Trophy!")

st.markdown("![Alt Text](https://media1.tenor.com/m/cnBtMqNDAYYAAAAd/soccer-celebration.gif)")